In [5]:
import os
os.chdir("..")

In [9]:
import json
import pandas as pd

# CONFIG CREATION

## Products

In [4]:
! gcloud auth application-default login



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=Pxlggx0ZWeuAebZQnwpdV4VcBa6mQQ&access_type=offline&code_challenge=1GZXXGtS8EKz1X0BEE-OqEECbuHmfQ4gTFim3mmYzbw&code_challenge_method=S256


Credentials saved to file: [/Users/benji-qu/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "data-359211" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


Updates are available for some Google Cloud CLI co

### Create unique categories

In [10]:
# new_dataset_name = "20250908"
# product_data = pd.read_json(f"gs://superlinked-benchmarks-internal/wayfair_wands_datasets/{new_dataset_name}/products.jsonl", lines=True)
product_data = pd.read_json(
    "/Users/benji-qu/Projects/wands-product-search/data/output/products.jsonl",
    lines=True
)
# product_data = data_prep(product_data, settings)

In [11]:
product_data.head()

,product_id,product_name,product_description,product_class,material,style,color,rating_count,average_rating,countryoforigin,price
0,product_1,vezina 65 '' rolled arm chesterfield loveseat,"in the endless world of sofa style options , t...",Sofas,"[foam, faux leather]",[traditional],"[brown, red]",4989,5.0,[China],2100
1,product_2,2 drawer standard dresser,"2 drawer standard dresser, Furniture / Bedroom...",Dressers & Chests,[wood],[traditional],"[white, yellow]",2511,5.0,[China],1600
2,product_3,peacock figurine,"peacock figurine, Décor & Pillows / Home Acces...",Decorative Objects,[metal],[traditional],[bronze],3107,4.3,[China],100
3,product_4,chenut rooster figurine,"chenut rooster figurine, Décor & Pillows / Hom...",Decorative Objects,[ceramic],[country],[gold],2456,5.0,[China],85
4,product_5,configrauble living room set,configurable living room set constructions hav...,Coffee & Cocktail Tables,"[foam, metal]",[country],[brown],2780,5.0,[USA],650


In [12]:
list_cols = ["material", "style", "color", "countryoforigin"]

unique_categories = {
    list_col: list(
        set([item for sublist in product_data[list_col].tolist() for item in sublist])
    )
    for list_col in list_cols
}

def get_unique_categories(series: pd.Series) -> list[str]:
    return series.dropna().unique().tolist()

unique_categories["product_class"] = get_unique_categories(
    product_data["product_class"]
)
with open("/Users/benji-qu/Projects/wands-product-search/data/output/unique_categories.json", "w") as f:
    json.dump(unique_categories, f, indent=2)


In [7]:
upload_text_to_gcs(
    json.dumps(unique_categories), config.bucket_name, f'wayfair_wands_datasets/{new_dataset_name}/unique_categories.json'
)

### Check unique categories

In [7]:
product_data = pd.read_json(
            f"gs://{config.bucket_name}/{config.product_dataset_path}", lines=True
        )

unique_categories = json.loads(
    download_text_from_gcs(
        bucket_name=config.bucket_name, file_path=config.unique_categories_path
    )
)

In [8]:
desc = product_data[
    ["rating_count", "average_rating", "price"]
].describe(percentiles=[0.1 * x for x in range(1, 10)] + [0.95, 0.98, 0.99])
desc

,rating_count,average_rating,price
count,10100.000000,10100.000000,10100.000000
mean,2934.494554,4.446574,1044.297525
std,3977.781407,0.424737,969.870418
min,45.000000,3.000000,0.000000
10%,199.900000,4.000000,125.000000
20%,433.800000,4.000000,250.000000
30%,873.400000,4.500000,375.000000
40%,1399.600000,4.500000,525.000000
50%,2092.500000,4.500000,725.000000
60%,2926.800000,4.500000,1000.000000


## Queries

In [17]:
queries = pd.DataFrame(
    json.loads(
        download_text_from_gcs(config.bucket_name, config.query_dataset_path)
    )
).set_index("query_id")

In [18]:
param_names = sorted(
    list(
        set(
            [
                item
                for sublist in [f.keys() for f in queries["query_params"].tolist()]
                for item in sublist
            ]
        )
    )
)

In [19]:
param_names_f = []
for name in param_names:
    if "weight" in name:
        param_names_f.append(name)
    else:
        param_names_f.append(name)

In [20]:
param_names_f

['color',
 'color_negated',
 'material',
 'material_negated',
 'price_max',
 'price_weight',
 'product_class',
 'product_description',
 'rating_count_min',
 'rating_count_weight',
 'rating_min',
 'rating_weight',
 'style',
 'style_negated']

## Ranked results

In [9]:
ranked_results = json.loads(download_text_from_gcs(config.bucket_name, config.ground_truth_input_path))

In [10]:
len_ser = pd.Series({k: len(v) for k, v in ranked_results.items()})

In [11]:
len_ser.describe(percentiles=[0.1 * x for x in range(1, 10)])

count    110.000000
mean       9.727273
std        0.975941
min        5.000000
10%        9.900000
20%       10.000000
30%       10.000000
40%       10.000000
50%       10.000000
60%       10.000000
70%       10.000000
80%       10.000000
90%       10.000000
max       10.000000
dtype: float64

In [12]:
len_ser

simple-001     5
simple-002     5
simple-003    10
simple-004     8
simple-005     9
              ..
synth-005     10
synth-006     10
synth-008     10
synth-009      9
synth-010     10
Length: 109, dtype: int64

# QUERY PART

In [ ]:
queries[0]

In [ ]:
input_queries = pd.DataFrame.from_dict(queries).set_index("query_id")

In [ ]:
eval_res = download_pickle_from_gcs(config.bucket_name, config.eval_result_output_path)

In [ ]:
[elem for elem in eval_res[0]]

In [ ]:
eval_res = [
    [elem for elem in f] + [input_queries.loc[f[0], "query_type"]] for f in eval_res
]

In [ ]:
eval_res[0]

In [ ]:
gt_eval = download_pickle_from_gcs(
    config.bucket_name, config.eval_ground_truth_output_path
)

In [ ]:
evaluate_rankings(eval_res, gt_eval, 10)

## Query surgery

In [11]:
queries = json.load(open("./data/output/queries.json"))

In [13]:
for i, query in enumerate(queries):
    for k, v in query["query_params"].items():
        if "weight" in k:
            queries[i]["query_params"][k] = v / 2

In [15]:
json.dump(queries, open("./data/output/queries.json", "w"))